### grp

# Spark: The Definitive Guide

## PART 1: Gentle Overview of Big Data and Spark

## dataPaths

In [35]:
import findspark
findspark.init()
import pyspark
sc = pyspark.SparkContext(master='spark://164.52.193.152:7077', appName='test')


In [34]:
#sc.stop()

In [36]:
from pyspark.sql import SparkSession
spark = SparkSession(sc)

In [37]:
flightData2015 = 'data/flight-data/csv/2015-summary.csv'
retailDataDay = 'data/retail-data/by-day/'

## _Chapter #1 - What is Apache Spark?_

-  Unified computing engine for parallel data processing distributed across clusters (machine nodes)   
<br>
    -  **Structured APIs**:
        -  Datasets
        -  DataFrames
        -  SQL   
        <br>
    -  **Unstructured APIs**:
        -  RDDs   
        <br>
    -  **Libraries**:
        -  Structured Streaming
        -  Machine Learning
        -  Graph   
        <br>
    -  **Resource Manager**:
        -  Local
        -  Standalone (Cluster)
        -  YARN (Cluster)
        -  Mesos (Cluster)   
        <br>
    -  **Language APIs**:
        -  Scala
        -  Java
        -  Python
        -  SQL
        -  R

## _Chapter #2 - A Gentle Introduction to Spark_

-  **Spark Applications**:
    -  **Driver** (heart of Spark Application during application's lifecycle):
        -  maintains information about Spark Application
        -  responds to user's program / input
        -  distributes and schedules work across executors   
        <br>
    -  **Executors**:
        -  executes work (code) assigned by driver
        -  reports state of work execution back to driver node   
        <br>
    -  **SparkSession**:
        -  entry point that manages Spark Application via driver process   
        <br>
    -  **DataFrames**:
        -  represents a table of data with rows and columns
        -  compiled in a schema that defines the column labels and data types   
        <br>
    -  **Partitions**:
        -  chunks of data distributed across cluster for parallel execution
        -  in addition, a collection of rows sitting on one physical machine in cluster
        -  parallelism = partitions = executors (x: 1 partition / 1,000 executors = parallelism of 1; 1,000 paritions / 1 executors = parallelism of 1)   
        <br>
    -  **Lazy Evaluation**:
        -  bundles plan of transformations on source data into DAG then triggers DAG on action
        -  molds a logical plan into a pysical plan that will run across cluster   
        <br>
    -  **Transformations**:
         -  data manipulations and modifications   
            <br>
             -  **Narrow Transformations** (1 to 1):
                 -  each input partition will contribute to only one output partition
                 -  no dependencies => 1 parent w/ 1 child
                 -  ex: filter, maps   
                <br>
             -  **Wide Transformations** (1 to N):
                  -  "aka" shuffle
                  -  many dependencies => 1 parent w/ many children
                  -  each input partition will contribute to many output partitions across the cluster
                  -  **when a shuffle occurs Spark writes the results to disk** ex: spark.sql.shuffle.partitions
                  -  ex: aggregations, joins, groupings   
       <br>           
    -  **Actions**:
        -  triggers the series of transformations into a spark job
            -  types:
                -  view data in the console
                -  collect data
                -  write to output data sources

## **Additional Definitions**:
   -  Spark Job (represents set of transformations triggered by an individual action)
   -  Schema Inference (have Spark best guess the schema of data) ***triggers Spark Job when scanning through data***
   -  Spark-Submit (launches application code to a cluster)
   -  Catalyst (planning and processing of work engine)

### _Spark contains separate Python and R processes hence when using Spark from Python or R language API the Python or R code is transaled into code that Spark can run on the executor JVMs_

### _Chapter #2 - Exercises_

In [38]:
flightDataDF2015 = spark\
.read\
.option("inferSchema", "true")\
.option("header", "true")\
.csv(flightData2015)

In [39]:
flightDataDF2015.rdd.getNumPartitions()

1

In [40]:
flightDataDF2015.take(3)

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Ireland', count=344)]

-  **Explain Plan**:
    -  displays DFs lineage

In [66]:
flightDataDF2015.sort("count").explain()

== Physical Plan ==
*Sort [count#192 ASC NULLS FIRST], true, 0
+- Exchange rangepartitioning(count#192 ASC NULLS FIRST, 5)
   +- *FileScan csv [DEST_COUNTRY_NAME#190,ORIGIN_COUNTRY_NAME#191,count#192] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/home/spark/spark-2.2.0-bin-hadoop2.7/plutopy/plutopy/sparkTheDefinitiveGu..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,ORIGIN_COUNTRY_NAME:string,count:int>


-  **SPARK.SQL.SHUFFLE.PARTITIONS**:
    -  by default, there are 200 shuffle partitions

In [67]:
spark.conf.set("spark.sql.shuffle.partitions", "5")

In [70]:
flightDataDF2015.sort("count").explain(True)

== Parsed Logical Plan ==
'Sort ['count ASC NULLS FIRST], true
+- Relation[DEST_COUNTRY_NAME#190,ORIGIN_COUNTRY_NAME#191,count#192] csv

== Analyzed Logical Plan ==
DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string, count: int
Sort [count#192 ASC NULLS FIRST], true
+- Relation[DEST_COUNTRY_NAME#190,ORIGIN_COUNTRY_NAME#191,count#192] csv

== Optimized Logical Plan ==
Sort [count#192 ASC NULLS FIRST], true
+- Relation[DEST_COUNTRY_NAME#190,ORIGIN_COUNTRY_NAME#191,count#192] csv

== Physical Plan ==
*Sort [count#192 ASC NULLS FIRST], true, 0
+- Exchange rangepartitioning(count#192 ASC NULLS FIRST, 5)
   +- *FileScan csv [DEST_COUNTRY_NAME#190,ORIGIN_COUNTRY_NAME#191,count#192] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/home/spark/spark-2.2.0-bin-hadoop2.7/plutopy/plutopy/sparkTheDefinitiveGu..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,ORIGIN_COUNTRY_NAME:string,count:int>


In [71]:
flightDataDF2015.createOrReplaceTempView("flight_data_2015")

In [72]:
sqlWay = spark\
.sql("""
select dest_country_name, count(1)
from flight_data_2015
group by dest_country_name
""")

In [73]:
dataFrameWay = flightDataDF2015\
.groupBy("dest_country_name")\
.count()

In [74]:
sqlWay.explain()
dataFrameWay.explain()

== Physical Plan ==
*HashAggregate(keys=[dest_country_name#190], functions=[count(1)])
+- Exchange hashpartitioning(dest_country_name#190, 5)
   +- *HashAggregate(keys=[dest_country_name#190], functions=[partial_count(1)])
      +- *FileScan csv [DEST_COUNTRY_NAME#190] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/home/spark/spark-2.2.0-bin-hadoop2.7/plutopy/plutopy/sparkTheDefinitiveGu..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string>
== Physical Plan ==
*HashAggregate(keys=[dest_country_name#190], functions=[count(1)])
+- Exchange hashpartitioning(dest_country_name#190, 5)
   +- *HashAggregate(keys=[dest_country_name#190], functions=[partial_count(1)])
      +- *FileScan csv [DEST_COUNTRY_NAME#190] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/home/spark/spark-2.2.0-bin-hadoop2.7/plutopy/plutopy/sparkTheDefinitiveGu..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string>


In [76]:
from pyspark.sql.functions import desc

In [77]:
flightDataDF2015\
.groupBy("dest_country_name")\
.sum("count")\
.withColumnRenamed("sum(count)", "destination_total")\
.sort(desc("destination_total"))\
.limit(5)\
.explain()

== Physical Plan ==
TakeOrderedAndProject(limit=5, orderBy=[destination_total#8121L DESC NULLS LAST], output=[dest_country_name#190,destination_total#8121L])
+- *HashAggregate(keys=[dest_country_name#190], functions=[sum(cast(count#192 as bigint))])
   +- Exchange hashpartitioning(dest_country_name#190, 5)
      +- *HashAggregate(keys=[dest_country_name#190], functions=[partial_sum(cast(count#192 as bigint))])
         +- *FileScan csv [DEST_COUNTRY_NAME#190,count#192] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/home/spark/spark-2.2.0-bin-hadoop2.7/plutopy/plutopy/sparkTheDefinitiveGu..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,count:int>


In [78]:
flightDataDF2015\
.groupBy("dest_country_name")\
.sum("count")\
.withColumnRenamed("sum(count)", "destination_total")\
.sort(desc("destination_total"))\
.limit(3)\
.show()

+-----------------+-----------------+
|dest_country_name|destination_total|
+-----------------+-----------------+
|    United States|           411352|
|           Canada|             8399|
|           Mexico|             7140|
+-----------------+-----------------+



## _Chapter #3 - A Tour of Spark's Toolset_

### _Chapter #3 - Exercise (Structured Streaming)_

-  **Structured Streaming**:
    -  read streams
    -  window functions
    -  triggers
    -  write streams

In [79]:
staticDataFrame = spark\
.read\
.format("csv")\
.option("header", "true")\
.option("inferSchema", "true")\
.load(retailDataDay)

In [80]:
staticDataFrame.createOrReplaceTempView("retail_data")
staticSchema = staticDataFrame.schema

In [81]:
from pyspark.sql.functions import window, column, desc, col

In [82]:
staticDataFrame\
.selectExpr(\
           "CustomerId",
           "(UnitPrice * Quantity) as total_cost",\
           "InvoiceDate")\
.groupBy(\
        col("CustomerId"), window(col("InvoiceDate"), "1 day"))\
.sum("total_cost")\
.show(3, truncate=False)

+----------+---------------------------------------------+------------------+
|CustomerId|window                                       |sum(total_cost)   |
+----------+---------------------------------------------+------------------+
|14075.0   |[2011-12-05 00:00:00.0,2011-12-06 00:00:00.0]|316.78000000000003|
|18180.0   |[2011-12-05 00:00:00.0,2011-12-06 00:00:00.0]|310.73            |
|15358.0   |[2011-12-05 00:00:00.0,2011-12-06 00:00:00.0]|830.0600000000003 |
+----------+---------------------------------------------+------------------+
only showing top 3 rows



In [83]:
streamingDataFrame = spark\
.readStream\
.schema(staticSchema)\
.option("maxFilesPerTrigger", 1)\
.format("csv")\
.option("header", "true")\
.load(retailDataDay)

In [84]:
print(streamingDataFrame.isStreaming)

True


In [87]:
purchaseByCustomerPerHour = streamingDataFrame\
.selectExpr(\
           "CustomerId",
           "(UnitPrice * Quantity) as total_cost",\
           "InvoiceDate")\
.groupBy(\
        col("CustomerId"), window(col("InvoiceDate"), "1 day"))\
.sum("total_cost")

In [88]:
purchaseByCustomerPerHour.writeStream\
.format("memory")\
.queryName("customer_purchases")\
.outputMode("complete")\
.start()


IllegalArgumentException: 'Cannot start query with name customer_purchases as a query with that name is already active'

In [89]:
from time import sleep
sleep(10)

In [90]:
spark.sql("""
select *
from customer_purchases
order by 'sum(total_cost)' desc""")\
.show(3, truncate=False)

+----------+---------------------------------------------+------------------+
|CustomerId|window                                       |sum(total_cost)   |
+----------+---------------------------------------------+------------------+
|15237.0   |[2011-12-08 00:00:00.0,2011-12-09 00:00:00.0]|83.6              |
|16811.0   |[2011-12-05 00:00:00.0,2011-12-06 00:00:00.0]|232.3             |
|12921.0   |[2011-03-30 00:00:00.0,2011-03-31 00:00:00.0]|-87.30000000000001|
+----------+---------------------------------------------+------------------+
only showing top 3 rows



### _Chapter #3 - Exercise (Machine Learning)_

-  **Machine Learning**:
    -  numerical representation
    -  data cleansing
    -  train / test split
    -  feature engineering (index, encode, vector assemble)

In [91]:
from pyspark.sql.functions import date_format, col

In [92]:
preppedDataFrame = staticDataFrame\
.na.fill(0)\
.withColumn("day_of_week", date_format(col("InvoiceDate"), "EEEE"))\
.coalesce(5)

In [93]:
trainDF = preppedDataFrame\
.where("InvoiceDate < '2011-07-01'")
testDF = preppedDataFrame\
.where("InvoiceDate >= '2011-07-01'")

In [94]:
print(trainDF.count())
print(testDF.count())

245903
296006


In [95]:
for i in trainDF.take(3): print(i)

Row(InvoiceNo='537226', StockCode='22811', Description='SET OF 6 T-LIGHTS CACTI ', Quantity=6, InvoiceDate=datetime.datetime(2010, 12, 6, 8, 34), UnitPrice=2.95, CustomerID=15987.0, Country='United Kingdom', day_of_week='Monday')
Row(InvoiceNo='537226', StockCode='21713', Description='CITRONELLA CANDLE FLOWERPOT', Quantity=8, InvoiceDate=datetime.datetime(2010, 12, 6, 8, 34), UnitPrice=2.1, CustomerID=15987.0, Country='United Kingdom', day_of_week='Monday')
Row(InvoiceNo='537226', StockCode='22927', Description='GREEN GIANT GARDEN THERMOMETER', Quantity=2, InvoiceDate=datetime.datetime(2010, 12, 6, 8, 34), UnitPrice=5.95, CustomerID=15987.0, Country='United Kingdom', day_of_week='Monday')


In [100]:
from pyspark.ml.feature import StringIndexer

In [101]:
indexer = StringIndexer()\
.setInputCol("day_of_week")\
.setOutputCol("day_of_week_index")

In [102]:
indexer.fit(trainDF).transform(testDF).select("day_of_week", "day_of_week_index").show(3, truncate=False)

+-----------+-----------------+
|day_of_week|day_of_week_index|
+-----------+-----------------+
|Monday     |2.0              |
|Monday     |2.0              |
|Monday     |2.0              |
+-----------+-----------------+
only showing top 3 rows



In [103]:
from pyspark.ml.feature import OneHotEncoder

In [104]:
encoder = OneHotEncoder()\
.setInputCol("day_of_week_index")\
.setOutputCol("day_of_week_encoded")

In [105]:
from pyspark.ml.feature import VectorAssembler

In [106]:
vectorAssembler = VectorAssembler()\
.setInputCols(["UnitPrice", "Quantity", "day_of_week_encoded"])\
.setOutputCol("features")

In [107]:
from pyspark.ml import Pipeline

In [108]:
transformationPipeline = Pipeline()\
.setStages([indexer, encoder, vectorAssembler]) # feature engineering to prepare for learning algorithm

In [109]:
fittedPipeline = transformationPipeline.fit(trainDF)

In [110]:
transformedTraining = fittedPipeline.transform(trainDF)

In [111]:
transformedTraining.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: timestamp (nullable = true)
 |-- UnitPrice: double (nullable = false)
 |-- CustomerID: double (nullable = false)
 |-- Country: string (nullable = true)
 |-- day_of_week: string (nullable = true)
 |-- day_of_week_index: double (nullable = true)
 |-- day_of_week_encoded: vector (nullable = true)
 |-- features: vector (nullable = true)



In [112]:
transformedTraining.select("UnitPrice", "Quantity", \
                      "day_of_week", "day_of_week_index", \
                      "day_of_week_encoded", "features")\
.show(3, truncate=False)

+---------+--------+-----------+-----------------+-------------------+--------------------------+
|UnitPrice|Quantity|day_of_week|day_of_week_index|day_of_week_encoded|features                  |
+---------+--------+-----------+-----------------+-------------------+--------------------------+
|2.95     |6       |Monday     |2.0              |(5,[2],[1.0])      |(7,[0,1,4],[2.95,6.0,1.0])|
|2.1      |8       |Monday     |2.0              |(5,[2],[1.0])      |(7,[0,1,4],[2.1,8.0,1.0]) |
|5.95     |2       |Monday     |2.0              |(5,[2],[1.0])      |(7,[0,1,4],[5.95,2.0,1.0])|
+---------+--------+-----------+-----------------+-------------------+--------------------------+
only showing top 3 rows



In [113]:
transformedTraining.cache()

DataFrame[InvoiceNo: string, StockCode: string, Description: string, Quantity: int, InvoiceDate: timestamp, UnitPrice: double, CustomerID: double, Country: string, day_of_week: string, day_of_week_index: double, day_of_week_encoded: vector, features: vector]

In [114]:
print(transformedTraining.count())

245903


-  **kMeans**:
    -  "k" centers are assigned to data points
    -  points are assigned to a class and center points (centroid) are computed

In [115]:
from pyspark.ml.clustering import KMeans

In [116]:
kmeans = KMeans()\
.setK(20)\
.setSeed(1)

In [117]:
kmModel = kmeans.fit(transformedTraining)

In [118]:
kmModel.computeCost(transformedTraining)

84553739.96537484

In [119]:
transformedTest = fittedPipeline.transform(testDF)

In [120]:
kmModel.computeCost(transformedTest)

517507094.7222117

### grp